# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [13]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,colonia,50.9333,6.9500,11.57,81,75,9.26,DE,1707267634
1,1,iqaluit,63.7506,-68.5145,-21.15,59,0,7.20,CA,1707267725
2,2,sandnessjoen,66.0217,12.6316,-15.71,92,100,5.89,NO,1707267725
3,3,keflavik,64.0049,-22.5624,-1.05,74,20,10.29,IS,1707267726
4,4,cabedelo,-6.9811,-34.8339,28.37,78,15,4.32,BR,1707267726


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [36]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points(
    x='Lng',
    y='Lat',
    color='City',
    #hover_cols=['City', 'Humidity'],
    #colorbar=True,
    size='Humidity',
    title='City Map with Humidity',
    xlabel='Longitude',
    ylabel='Latitude',
    #cmap='viridis',  # Choose a color map
    alpha=0.7,  # Set transparency
    frame_height=500,  # Set frame height
    geo = True,
    tiles = "OSM"
)

# Display the map
city_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [15]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_cities_df = city_data_df[(city_data_df['Humidity'] > 50) & (city_data_df['Max Temp'] < 20)]

# Drop any rows with null values
filtered_cities_df = filtered_cities_df.dropna()

# Display sample data
filtered_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,colonia,50.9333,6.9500,11.57,81,75,9.26,DE,1707267634
1,1,iqaluit,63.7506,-68.5145,-21.15,59,0,7.20,CA,1707267725
2,2,sandnessjoen,66.0217,12.6316,-15.71,92,100,5.89,NO,1707267725
3,3,keflavik,64.0049,-22.5624,-1.05,74,20,10.29,IS,1707267726
5,5,hermanus,-34.4187,19.2345,19.89,83,0,6.69,ZA,1707267469


### Step 3: Create a new DataFrame called `hotel_df`.

In [16]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,colonia,DE,50.9333,6.9500,81,
1,iqaluit,CA,63.7506,-68.5145,59,
2,sandnessjoen,NO,66.0217,12.6316,92,
3,keflavik,IS,64.0049,-22.5624,74,
4,cabedelo,BR,-6.9811,-34.8339,78,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [41]:
# Set parameters to search for a hotel
radius = 10000
params = {
    #"radius": radius,
    "categories": "accommodation.hotel",
     "apiKey": geoapify_key,
    "limit": 15,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params).json()
    
    # Convert the API response to JSON format
    #name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
colonia - nearest hotel: Wasserturm Hotel Cologne
iqaluit - nearest hotel: Frobisher Inn
sandnessjoen - nearest hotel: Scandic Syv Søstre
keflavik - nearest hotel: Núpan Deluxe
cabedelo - nearest hotel: No hotel found
hermanus - nearest hotel: Aloe guest house
adamstown - nearest hotel: No hotel found
lyasny - nearest hotel: гасцініца Старт
grytviken - nearest hotel: No hotel found
zhaqsy - nearest hotel: Гостиница-кафе 24/7
port-aux-francais - nearest hotel: Keravel
blackmans bay - nearest hotel: Villa Howden
nadi - nearest hotel: President Hotel
utrik - nearest hotel: No hotel found
niquero - nearest hotel: Hotel Niquero
saipan - nearest hotel: Chalan Kanoa Beach Hotel
bengkulu - nearest hotel: Dena Hotel
alofi - nearest hotel: Matavai Resort
holualoa - nearest hotel: Kona Hotel
tiksi - nearest hotel: Арктика
tazovsky - nearest hotel: Тазовчанка
selfoss - nearest hotel: Miðgarður
balotra - nearest hotel: No hotel found
isafjordur - nearest hotel: Hótel Horn
vads

borogontsy - nearest hotel: No hotel found
bodo - nearest hotel: Zefyr hotell
port macquarie - nearest hotel: The Observatory
bodaybo - nearest hotel: НОЙ
juegang - nearest hotel: No hotel found
menongue - nearest hotel: No hotel found
durban - nearest hotel: Tudor
fort bragg - nearest hotel: Airborne Inn Lodging
cedral - nearest hotel: No hotel found
nemuro - nearest hotel: お宿エクハシ
uturoa - nearest hotel: Hawaiki Nui hotel
darwin - nearest hotel: Mantra Pantanas Darwin
hawaiian paradise park - nearest hotel: No hotel found
thanh hoa - nearest hotel: Thanh Hoa Hotel
samana - nearest hotel: Hotel Cotubanamá
vorgashor - nearest hotel: София
ixtapa - nearest hotel: Ma. Cristina
pirojpur - nearest hotel: হোটেল অবকাশ
kavaratti - nearest hotel: No hotel found
kiama - nearest hotel: Nova Kiama
nova sintra - nearest hotel: Residência Ka Dencho
laguna - nearest hotel: Holiday Inn Express & Suites
port elizabeth - nearest hotel: Waterford Hotel
qarauyl - nearest hotel: No hotel found
buulobarde -

udaipur - nearest hotel: Pannadhay Palace
roswell - nearest hotel: Best Western Plus Hotel
ghadamis - nearest hotel: فندق عين الفارس
mokrous - nearest hotel: No hotel found
constantia - nearest hotel: Hotel Chérica
alamogordo - nearest hotel: Days Inn Alamogordo
east london - nearest hotel: No hotel found
ikwiriri - nearest hotel: No hotel found
mournies - nearest hotel: Oasis
ouro preto - nearest hotel: Hotel Nossa Senhora Aparecida
baoding - nearest hotel: Homeinn
bucheon-si - nearest hotel: 모리호텔
ta'izz - nearest hotel: سكن
stung treng - nearest hotel: សណ្ធាគារសេកុងស្គារ
garden city - nearest hotel: Quality Inn & Suites
maine soroa - nearest hotel: Maïné-Soroa
port-vila - nearest hotel: Holiday Inn Resort Vanuatu
remire-montjoly - nearest hotel: Complexe Belova
cestos city - nearest hotel: No hotel found
zyryanovsk - nearest hotel: У Брилина
kudymkar - nearest hotel: No hotel found
mandalgovi - nearest hotel: Б
querecotillo - nearest hotel: Hospedaje Rey & Ba
san patricio - nearest h

aripuana - nearest hotel: No hotel found
zhongxiang - nearest hotel: No hotel found
savanna-la-mar - nearest hotel: Kibo Hotel
tromso - nearest hotel: Clarion Collection Hotel Aurora
mananara - nearest hotel: Chez Roger
sinjai - nearest hotel: Hotel Sinjai
funadhoo - nearest hotel: JW Marriott Maldives Resort & Spa
cedeira - nearest hotel: Hospedería Casa Cordobelas
chul'man - nearest hotel: Чорон
shibirghan - nearest hotel: Attayee Hotel and Guesthouse
al kharj - nearest hotel: شقق فندقية
vilyuysk - nearest hotel: Мария
vorkuta - nearest hotel: "Мегаполис"


,City,Country,Lat,Lng,Humidity,Hotel Name
0,colonia,DE,50.9333,6.9500,81,Wasserturm Hotel Cologne
1,iqaluit,CA,63.7506,-68.5145,59,Frobisher Inn
2,sandnessjoen,NO,66.0217,12.6316,92,Scandic Syv Søstre
3,keflavik,IS,64.0049,-22.5624,74,Núpan Deluxe
4,cabedelo,BR,-6.9811,-34.8339,78,No hotel found
...,...,...,...,...,...,...
580,chul'man,RU,56.8479,124.9104,94,Чорон
581,shibirghan,AF,36.6676,65.7529,34,Attayee Hotel and Guesthouse
582,al kharj,SA,24.1550,47.3346,42,شقق فندقية
583,vilyuysk,RU,63.7553,121.6247,100,Мария


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [45]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points(
    x='Lng',
    y='Lat',
    c='City',
    hover_cols=['Lng', 'Lat', 'City', 'Country', 'Humidity', 'Hotel Name'],
    #colorbar=True,
    size='Humidity',
    title='Hotels By City',
    xlabel='Longitude',
    ylabel='Latitude',
    #cmap='viridis',  # Choose a color map
    alpha=0.7,  # Set transparency
    frame_height=500,  # Set frame height
    geo = True,
    tiles = "OSM"
)

# Display the map
city_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)